<a href="https://colab.research.google.com/github/Sarita-Salunke/EVA2Course/blob/main/S4_Assignment_Sarita.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [30]:

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, 3, padding=1) #input -? OUtput? RF
        self.conv1_bn = nn.BatchNorm2d(10)
        self.conv2 = nn.Conv2d(10, 15, 3, padding=1)
        self.conv2_bn = nn.BatchNorm2d(15)
        self.pool1 = nn.MaxPool2d(2,2)
        self.conv3 = nn.Conv2d(15, 20, 3, padding=1)
        self.conv3_bn = nn.BatchNorm2d(20)
        self.conv4 = nn.Conv2d(20, 30, 3, padding=1)
        self.conv4_bn = nn.BatchNorm2d(30)
        self.pool2 = nn.MaxPool2d(2,2)
        self.conv5 = nn.Conv2d(30, 35, 3)
        self.conv5_bn = nn.BatchNorm2d(35)
        self.pool3 = nn.AvgPool2d(5,5)
       
        self.conv7 = nn.Conv2d(35, 10, 1)


    def forward(self, x):
        x = self.pool1(self.conv2_bn(F.relu(self.conv2(self.conv1_bn((F.relu(self.conv1(x))))))))
        x = self.pool2(self.conv4_bn(F.relu(self.conv4(self.conv3_bn((F.relu(self.conv3(x))))))))
        x = self.pool3((F.relu(self.conv5(x))))

        x = self.conv7(x)
       

        
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [31]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 10, 28, 28]             100
       BatchNorm2d-2           [-1, 10, 28, 28]              20
            Conv2d-3           [-1, 15, 28, 28]           1,365
       BatchNorm2d-4           [-1, 15, 28, 28]              30
         MaxPool2d-5           [-1, 15, 14, 14]               0
            Conv2d-6           [-1, 20, 14, 14]           2,720
       BatchNorm2d-7           [-1, 20, 14, 14]              40
            Conv2d-8           [-1, 30, 14, 14]           5,430
       BatchNorm2d-9           [-1, 30, 14, 14]              60
        MaxPool2d-10             [-1, 30, 7, 7]               0
           Conv2d-11             [-1, 35, 5, 5]           9,485
        AvgPool2d-12             [-1, 35, 1, 1]               0
           Conv2d-13             [-1, 10, 1, 1]             360
Total params: 19,610
Trainable params: 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:32: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [32]:


torch.manual_seed(1)
batch_size = 16

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [33]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [34]:
torch.manual_seed(1)
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 20):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/3750 [00:00<?, ?it/s]


Test set: Average loss: 0.0339, Accuracy: 9875/10000 (99%)



  0%|          | 0/3750 [00:00<?, ?it/s]


Test set: Average loss: 0.0327, Accuracy: 9881/10000 (99%)



  0%|          | 0/3750 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0245, Accuracy: 9936/10000 (99%)



  0%|          | 0/3750 [00:00<?, ?it/s]


Test set: Average loss: 0.0188, Accuracy: 9942/10000 (99%)



loss=1.5555146092083305e-05 batch_id=3749: 100%|██████████| 3750/3750 [00:40<00:00, 93.47it/s]



Test set: Average loss: 0.0227, Accuracy: 9935/10000 (99%)

